## Data Collection 2

Collecting players' data

Imports

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

Removing cookies pop-up

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)
driver.get("https://www.whoscored.com/Statistics")
wait = WebDriverWait(driver, 5)
accept_cookies_button = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span')))
print("Cookies button found.")
accept_cookies_button.click()
driver.implicitly_wait(10)
wait.until(EC.invisibility_of_element((By.CSS_SELECTOR, '.accept-cookies')))
print("Cookies pop-up clicked")
wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#top-team-stats-options > li:nth-child(1) > a'))
    )

Scrolling necessary for scraping

In [ ]:
def scroll_to_percentage(percentage): #chatgpt
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_height = total_height * percentage
    driver.execute_script(f"window.scrollTo(0, {scroll_height});")
    time.sleep(3)

Extracting players data from the summary page

In [ ]:
def extract_players():
    player = [elem.text for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#statistics-table-summary .player-link span.iconize.iconize-icon-left')))]
    team_link = [elem.get_attribute('href') for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#statistics-table-summary #player-table-statistics-body .col12-lg-2 a.player-meta-data')))]
    mins = [elem.text for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#statistics-table-summary #player-table-statistics-body .minsPlayed')))]
    goals = [elem.text for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#statistics-table-summary #player-table-statistics-body .goal')))]    
    shots_pg = [elem.text for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#statistics-table-summary #player-table-statistics-body .shotsPerGame')))]
    pass_success = [elem.text for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#statistics-table-summary #player-table-statistics-body .passSuccess')))]
    assist = [elem.text for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#statistics-table-summary #player-table-statistics-body .assistTotal')))]
    aerials_won = [elem.text for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#statistics-table-summary #player-table-statistics-body .aerialWonPerGame')))]
    df_players = pd.DataFrame({
        'Player Name': player,
        'Team Link': team_link,
        'Minutes Played': mins,
        'Goals This Season': goals,
        'Assist This Season': assist,
        'Shots PG': shots_pg,
        'Pass Success%': pass_success,
        'Aerial Duels Won' : aerials_won
    })

    return df_players

Switching the page to expected goals

In [ ]:
xg_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#top-player-stats-options > li:nth-child(5) > a')))
xg_button.click()

Getting every page of the players' data

In [ ]:
df_players = pd.DataFrame()

for page_number in range(1,149):
    if page_number > 1:
        scroll_to_percentage(0.55)
        
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#statistics-paging-summary #next')))
            next_button.click()
            time.sleep(3)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#top-team-stats-options > li:nth-child(1) > a')))
            time.sleep(3)  # Asegúrate de que la página esté completamente cargada
        except Exception as e:
            print(f"Could not click the 'Next' button or the page did not load: {e}")     
            break
    # Extract data from the current page
    df_page_players = extract_players()
    df_players = pd.concat([df_players, df_page_players], ignore_index=True)

df_players

Getting each players expected goals

In [ ]:
df_players_xg = pd.DataFrame()

for page_number in range(1,149):
    if page_number > 1:
        scroll_to_percentage(0.55)
        
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#statistics-paging-xg #next')))
            next_button.click()
            time.sleep(3)  # Asegúrate de que la página esté completamente cargada
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#top-team-stats-options > li:nth-child(1) > a')))
            time.sleep(3)  # Asegúrate de que la página esté completamente cargada
        except Exception as e:
            print(f"Could not click the 'Next' button or the page did not load: {e}")
            

            break
        
    # Extract data from the current page
    df_page_players_xg = extract_players_Xg()
    df_players_xg = pd.concat([df_players_xg, df_page_players_xg], ignore_index=True)

df_players_xg

Cleaning players names

In [ ]:
def clean_name(name):
    return name.split('\n')[-1]
df_players_xg['Player Name'] = df_players_xg['Player Name'].apply(clean_name)

In [ ]:
driver.quit()